In [1]:
# coding:utf-8

import keras
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras import backend as K
import os
import glob
import matplotlib.gridspec as gridspec
import shutil
from keras.utils import plot_model
import cv2
from skimage import io
from skimage import transform
from scipy.misc import imresize
from skimage import io
import skimage

Using TensorFlow backend.


<class 'keras.engine.topology.Layer'>


In [2]:
# フォルダの中にある画像を順次読み込む
# カテゴリーは0から始める

#参考
#https://qiita.com/supersaiakujin/items/fc54116df9ca6958a68d
#http://testpy.hatenablog.com/entry/2017/06/02/001901
#https://qiita.com/zaburo/items/0b9db87d0a52191b164b
# 画像の反転と回転の実装、またはdata argumentation の実装を行う
X = []
Y = []
name_class = []
#下、後でコード量を減らす
# 対象Aの画像
rate = 0.2
label = 0
for dir in os.listdir("data/train"):
#     print(dir)
    if dir == ".DS_Store":
        continue
    name_class.append(dir)
    for picture in list_pictures("data/train/"+dir):
#         print(picture)
    #     img =load_img(picture, target_size=(64,64))
        img =imresize(skimage.io.imread(picture),(224,224), interp='nearest')
#         img =imresize(skimage.io.imread(picture),(64,64), interp='nearest')
    #     img_raw = skimage.io.imread
#         print(img_to_array(skimage.transform.rotate(img, angle=0, resize=False, center=None)).shape)
        for i in range(358,360):
            #回転のみ
            X.append(img_to_array(skimage.transform.rotate(img, angle=i, resize=False, center=None)))
            Y.append(label )
#             #回転と上下の反転
            X.append(img_to_array(cv2.flip(skimage.transform.rotate(img, angle=i, resize=False, center=None),0)))
            Y.append(label )
#             #回転と左右の反転
#             X.append(img_to_array(cv2.flip(skimage.transform.rotate(img, angle=i, resize=False, center=None),1)))
#             Y.append(label )
#             #拡大
            X.append(img_to_array(skimage.transform.warp(skimage.transform.rotate(img, angle=i, resize=False, center=None), skimage.transform.AffineTransform(scale=(1-rate, 1-rate), translation=(64,64)))))
            Y.append(label )
    label += 1

# # arrayに変換
# X.shape()
print(name_class)
X = np.asarray(X)
print(X.shape)
# print(X)
Y = np.asarray(Y)
# print(Y)

KeyboardInterrupt: 

In [ ]:
# 画素値を0から1の範囲に変換
X = X.astype('float32')
X = X / 255.0

# クラスの形式を変換
Y = np_utils.to_categorical(Y, 5)

# 学習用データとテストデータ
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=111)

In [ ]:
# # CNNを構築(kaggleコンペで高かった組み方)
# model = Sequential()

# model.add(Conv2D(32, (3, 3), padding='same',
#                  input_shape=X_train.shape[1:]))
# model.add(Activation('relu'))
# # model.add(Conv2D(32, (3, 3)))
# # model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), padding='same'))
# # model.add(Activation('relu'))
# # model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(5))       # クラスは2個
# model.add(Activation('softmax'))

# # コンパイル
# model.compile(loss='categorical_crossentropy',
#               optimizer='SGD',
#               metrics=['accuracy'])

# # 実行。出力はなしで設定(verbose=0)。
# history = model.fit(X_train, y_train, batch_size=5, epochs=50,
#                    validation_data = (X_test, y_test), verbose = 1)

# # plot_model(model, to_file="model.png", show_shapes=True)
# # https://qiita.com/agumon/items/ab2de98a3783e0a93e66

In [ ]:
# CNNを構成（VGG NET)
from keras.models import Sequential, Model, load_model
from keras import applications
model = Sequential()

initial_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=X_train.shape[1:])
last = initial_model.output

x = Flatten()(last)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
preds = Dense(5, activation='softmax')(x)

model = Model(initial_model.input, preds)
# model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.summary()
# コンパイル
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

# 実行。出力はなしで設定(verbose=0)。
history = model.fit(X_train, y_train, batch_size=5, epochs=5,
                   validation_data = (X_test, y_test), verbose = 1)

# plot_model(model, to_file="model.png", show_shapes=True)
# https://qiita.com/agumon/items/ab2de98a3783e0a93e66

In [ ]:
#VGG-19のModelで層を組んでいる

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(128,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(256,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(512,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(512,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512,(3, 3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))       # クラスは2個
model.add(Activation('softmax'))

model.summary()
# コンパイル
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

# 実行。出力はなしで設定(verbose=0)。
history = model.fit(X_train, y_train, batch_size=5, epochs=5,
                   validation_data = (X_test, y_test), verbose = 1)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['acc', 'val_acc'], loc='lower right') #グラフの凡例を用意している
plt.show()

In [ ]:
# テストデータに適用
predict_classes = model.predict_classes(X_test)

# マージ。yのデータは元に戻す
mg_df = pd.DataFrame({'predict': predict_classes, 'class': np.argmax(y_test, axis=1)})

# confusion matrix
pd.crosstab(mg_df['class'], mg_df['predict'])

In [ ]:
def train_and_evaluate_model(model,data,labels,data_test,labels_test):
# def train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test]):
    model.fit(data, labels, batch_size=32, epochs=30,verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
    return model.evaluate(data_test, labels_test, verbose=1)  # Evaluate the trained model on the test set!
#損失値と評価の値を算出している
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
# print(cross_val_score(model, image_list, label_list,cv=kfold)).mean()  
from sklearn.cross_validation import StratifiedKFold
label = np.r_[np.repeat(0,16), np.repeat(1,0)]
skf = StratifiedKFold(label, n_folds=15, shuffle=False)
total_score = 0
rep = 0
print("success")
for i, (train, test) in enumerate(skf):
    print("TRAIN:", train, "TEST:", test)
    total_score += train_and_evaluate_model(model, X_train[train], y_train[train], X_train[test], y_train[test])[1]
    rep+=1
# print(total_score/rep)
print("%.10f" % (total_score/rep))

In [ ]:
### 最初の3つのcellでとりあえず物体をopencvで検知するところまで行う
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import selectivesearch


In [ ]:
# -*- coding: utf-8 -*-
from __future__ import division

import skimage.io
import skimage.feature
import skimage.color
import skimage.transform
import skimage.util
import skimage.segmentation
import numpy


# "Selective Search for Object Recognition" by J.R.R. Uijlings et al.
#
#  - Modified version with LBP extractor for texture vectorization


def _generate_segments(im_orig, scale, sigma, min_size):
    """
        segment smallest regions by the algorithm of Felzenswalb and
        Huttenlocher
    """

    # open the Image
    im_mask = skimage.segmentation.felzenszwalb(
        skimage.util.img_as_float(im_orig), scale=scale, sigma=sigma,
        min_size=min_size)
#     plt.imshow(im_mask)
    # merge mask channel to the image as a 4th channel
    im_orig = numpy.append(
        im_orig, numpy.zeros(im_orig.shape[:2])[:, :, numpy.newaxis], axis=2)
    im_orig[:, :, 3] = im_mask
#     plt.imshow(im_orig)
    return im_orig


def _sim_colour(r1, r2):
    """
        calculate the sum of histogram intersection of colour
    """
    return sum([min(a, b) for a, b in zip(r1["hist_c"], r2["hist_c"])])


def _sim_texture(r1, r2):
    """
        calculate the sum of histogram intersection of texture
    """
    return sum([min(a, b) for a, b in zip(r1["hist_t"], r2["hist_t"])])


def _sim_size(r1, r2, imsize):
    """
        calculate the size similarity over the image
    """
    return 1.0 - (r1["size"] + r2["size"]) / imsize


def _sim_fill(r1, r2, imsize):
    """
        calculate the fill similarity over the image
    """
    bbsize = (
        (max(r1["max_x"], r2["max_x"]) - min(r1["min_x"], r2["min_x"]))
        * (max(r1["max_y"], r2["max_y"]) - min(r1["min_y"], r2["min_y"]))
    )
    return 1.0 - (bbsize - r1["size"] - r2["size"]) / imsize


def _calc_sim(r1, r2, imsize):
    return (_sim_colour(r1, r2) + _sim_texture(r1, r2)
            + _sim_size(r1, r2, imsize) + _sim_fill(r1, r2, imsize))


def _calc_colour_hist(img):
    """
        calculate colour histogram for each region

        the size of output histogram will be BINS * COLOUR_CHANNELS(3)

        number of bins is 25 as same as [uijlings_ijcv2013_draft.pdf]

        extract HSV
    """

    BINS = 25
    hist = numpy.array([])

    for colour_channel in (0, 1, 2):

        # extracting one colour channel
        c = img[:, colour_channel]

        # calculate histogram for each colour and join to the result
        hist = numpy.concatenate(
            [hist] + [numpy.histogram(c, BINS, (0.0, 255.0))[0]])

    # L1 normalize
    hist = hist / len(img)

    return hist


def _calc_texture_gradient(img):
    """
        calculate texture gradient for entire image

        The original SelectiveSearch algorithm proposed Gaussian derivative
        for 8 orientations, but we use LBP instead.

        output will be [height(*)][width(*)]
    """
    ret = numpy.zeros((img.shape[0], img.shape[1], img.shape[2]))

    for colour_channel in (0, 1, 2):
        ret[:, :, colour_channel] = skimage.feature.local_binary_pattern(
            img[:, :, colour_channel], 8, 1.0)

    return ret


def _calc_texture_hist(img):
    """
        calculate texture histogram for each region

        calculate the histogram of gradient for each colours
        the size of output histogram will be
            BINS * ORIENTATIONS * COLOUR_CHANNELS(3)
    """
    BINS = 10

    hist = numpy.array([])

    for colour_channel in (0, 1, 2):

        # mask by the colour channel
        fd = img[:, colour_channel]

        # calculate histogram for each orientation and concatenate them all
        # and join to the result
        hist = numpy.concatenate(
            [hist] + [numpy.histogram(fd, BINS, (0.0, 1.0))[0]])

    # L1 Normalize
    hist = hist / len(img)

    return hist


def _extract_regions(img):

    R = {}

    # get hsv image
    hsv = skimage.color.rgb2hsv(img[:, :, :3])

    # pass 1: count pixel positions
    for y, i in enumerate(img):

        for x, (r, g, b, l) in enumerate(i):

            # initialize a new region
            if l not in R:
                R[l] = {
                    "min_x": 0xffff, "min_y": 0xffff,
                    "max_x": 0, "max_y": 0, "labels": [l]}

            # bounding box
            if R[l]["min_x"] > x:
                R[l]["min_x"] = x
            if R[l]["min_y"] > y:
                R[l]["min_y"] = y
            if R[l]["max_x"] < x:
                R[l]["max_x"] = x
            if R[l]["max_y"] < y:
                R[l]["max_y"] = y

    # pass 2: calculate texture gradient
    tex_grad = _calc_texture_gradient(img)

    # pass 3: calculate colour histogram of each region
    for k, v in list(R.items()):

        # colour histogram
        masked_pixels = hsv[:, :, :][img[:, :, 3] == k]
        R[k]["size"] = len(masked_pixels / 4)
        R[k]["hist_c"] = _calc_colour_hist(masked_pixels)

        # texture histogram
        R[k]["hist_t"] = _calc_texture_hist(tex_grad[:, :][img[:, :, 3] == k])
#     print(R)
    return R


def _extract_neighbours(regions):

    def intersect(a, b):
        if (a["min_x"] < b["min_x"] < a["max_x"]
                and a["min_y"] < b["min_y"] < a["max_y"]) or (
            a["min_x"] < b["max_x"] < a["max_x"]
                and a["min_y"] < b["max_y"] < a["max_y"]) or (
            a["min_x"] < b["min_x"] < a["max_x"]
                and a["min_y"] < b["max_y"] < a["max_y"]) or (
            a["min_x"] < b["max_x"] < a["max_x"]
                and a["min_y"] < b["min_y"] < a["max_y"]):
            return True
        return False

    R = list(regions.items())
    neighbours = []
    for cur, a in enumerate(R[:-1]):
        for b in R[cur + 1:]:
            if intersect(a[1], b[1]):
                neighbours.append((a, b))

    return neighbours


def _merge_regions(r1, r2):
    new_size = r1["size"] + r2["size"]
    rt = {
        "min_x": min(r1["min_x"], r2["min_x"]),
        "min_y": min(r1["min_y"], r2["min_y"]),
        "max_x": max(r1["max_x"], r2["max_x"]),
        "max_y": max(r1["max_y"], r2["max_y"]),
        "size": new_size,
        "hist_c": (
            r1["hist_c"] * r1["size"] + r2["hist_c"] * r2["size"]) / new_size,
        "hist_t": (
            r1["hist_t"] * r1["size"] + r2["hist_t"] * r2["size"]) / new_size,
        "labels": r1["labels"] + r2["labels"]
    }
    return rt


def selective_search(
        im_orig, scale=1.0, sigma=0.8, min_size=50):
    '''Selective Search

    Parameters
    ----------
        im_orig : ndarray
           写真
        scale : int
            Free parameter. Higher means larger clusters in felzenszwalb segmentation.
        sigma : float
            Width of Gaussian kernel for felzenszwalb segmentation.
        min_size : int
            Minimum component size for felzenszwalb segmentation.
    Returns
    -------
        img : ndarray
            image with region label
            region label is stored in the 4th value of each pixel [r,g,b,(region)]
        regions : array of dict
            [
                {
                    'rect': (left, top, width, height),
                    'labels': [...],
                    'size': component_size
                },
                ...
            ]
    '''
    #3次元のデータでなければエラー文章を出力する
    assert im_orig.shape[2] == 3, "3ch image is expected"

    # load image and get smallest regions
    # region label is stored in the 4th value of each pixel [r,g,b,(region)]
    img = _generate_segments(im_orig, scale, sigma, min_size)

    plt.imshow(img)
    if img is None:
        return None, {}

    imsize = img.shape[0] * img.shape[1]
    R = _extract_regions(img)

    # extract neighbouring information
    neighbours = _extract_neighbours(R)

    # calculate initial similarities
    S = {}
    for (ai, ar), (bi, br) in neighbours:
        S[(ai, bi)] = _calc_sim(ar, br, imsize)

    # hierarchal search
    while S != {}:

        # get highest similarity
        i, j = sorted(S.items(), key=lambda i: i[1])[-1][0]

        # merge corresponding regions
        t = max(R.keys()) + 1.0
        R[t] = _merge_regions(R[i], R[j])

        # mark similarities for regions to be removed
        key_to_delete = []
        for k, v in list(S.items()):
            if (i in k) or (j in k):
                key_to_delete.append(k)

        # remove old similarities of related regions
        for k in key_to_delete:
            del S[k]

        # calculate similarity set with the new region
        for k in [a for a in key_to_delete if a != (i, j)]:
            n = k[1] if k[0] in (i, j) else k[0]
            S[(t, n)] = _calc_sim(R[t], R[n], imsize)

    regions = []
    for k, r in list(R.items()):
        regions.append({
            'rect': (
                r['min_x'], r['min_y'],
                r['max_x'] - r['min_x'], r['max_y'] - r['min_y']),
            'size': r['size'],
            'labels': r['labels']
        })

    return img, regions


In [ ]:

def main():
        # loading lena image
    img = cv2.imread("rice.png")

    # perform selective search
    img_lbl, regions = selective_search(
        img, scale=500, sigma=0.9, min_size=10)
#     print(regions)
#     img.show()
    candidates = set()
    for r in regions:
        # excluding same rectangle (with different segments)
        if r['rect'] in candidates:
            continue
        # excluding regions smaller than 2000 pixels
        if r['size'] < 2000:
            continue
        # distorted rects
        x, y, w, h = r['rect']
#         if w / h > 0.5 or h / w > 0.5:
        if w / h > 1.2 or h / w > 1.2:
            continue
        candidates.add(r['rect'])

    # draw rectangles on the original image
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 6))
    ax.imshow(img)
    X = []
    Y = []
#     print(candidates)
    for x, y, w, h in candidates:
#         print(x, y, w, h)
        rect = mpatches.Rectangle(
            (x, y), w, h, fill=False, edgecolor='red', linewidth=1)
        ax.add_patch(rect)
        X.append(img_to_array(imresize(img[y:y + h, x:x + w],(64,64), interp='nearest')))
        # cv2.imwrite('' + str(x) + '.jpg', img[y:y + h, x:x + w])
    X = np.asarray(X)
#     print(X.shape)
    predictions = model.predict(X)
        # round predictions
#     rounded = [round(X[0]) for x in predictions]
    print(predictions)
    for prediction in predictions:
        prediction = list(prediction)
        print(name_class[prediction.index(max(prediction))])
    plt.show()

if __name__ == "__main__":
    main()
